In [1]:
import pandas as pd

# 載入寶可夢資料
df = pd.read_json('pokemon.json')

# 取出目標欄位
X_train = df.iloc[:, :-1]
X_train.head()

,HP,Attack,Defense,SpecialAtk,SpecialDef
0,39,52,43,60,50
1,58,64,58,80,65
2,35,55,40,50,50
3,60,90,55,90,80
4,38,41,40,50,65


In [2]:
# 特徵標準化
from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()
scalar.fit(X_train)
X_train_std = scalar.transform(X_train)
X_train_std[:2, :]

array([[-1.15734402, -0.72671047, -0.84787115, -0.47156812, -0.78920593],
       [-0.35171516, -0.33802419, -0.24169379,  0.11647846, -0.17526249]])

In [3]:
# 利用 Hierarchical Clustering 進行分群，除以下參數設定外，其餘為預設值
# #############################################################################
# n_clusters=4, affinity='euclidean', linkage='ward'
# #############################################################################
from sklearn.cluster import AgglomerativeClustering

model = AgglomerativeClustering(n_clusters=4, 
                                affinity='euclidean', 
                                linkage='ward')
model = model.fit(X_train_std)

In [4]:
# 計算每一群的個數
num_lst = model.labels_.tolist()
for i in range(len(set(num_lst))):
    print('Cluster', i, '個數:', num_lst.count(i))

Cluster 0 個數: 76
Cluster 1 個數: 15
Cluster 2 個數: 49
Cluster 3 個數: 16


In [5]:
# 找到 Speed 有遺漏值的兩隻寶可夢，並填入組內平均
df['cluster'] = model.labels_

for i in df.index:
    if pd.isna(df.iloc[i, -2]):
        print(df.iloc[i,:].tolist())
        poke = df[df['cluster']==df.iloc[i, -1]].mean()
        print('Speed = %d' % round(poke['Speed'], 0))

[60.0, 48.0, 45.0, 43.0, 90.0, nan, 0.0]
Speed = 58
[70.0, 75.0, 60.0, 105.0, 60.0, nan, 2.0]
Speed = 92
